# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(['timestamp'], axis=1)
new_df.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [6]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {
    'n_factors': [20, 50, 100],
    'reg_all': [0.02, 0.05, 0.1]
}
gs_svd = GridSearchCV(SVD, param_grid=param_grid, n_jobs=-1)
gs_svd.fit(data)

In [7]:
# print out optimal parameters for SVD after GridSearch
print(f"grid search best score:\n\t{gs_svd.best_score}")
print(f"best params:\n\t{gs_svd.best_params}")

grid search best score:
	{'rmse': 0.8692759710639336, 'mae': 0.6681980146244146}
best params:
	{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [8]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
cv_knn_basic

{'test_rmse': array([0.9754431 , 0.97513365, 0.97647421, 0.97396211, 0.96707923]),
 'test_mae': array([0.75306096, 0.75219953, 0.75407193, 0.75397136, 0.74496152]),
 'fit_time': (1.1758737564086914,
  1.1576571464538574,
  1.1224555969238281,
  0.9199132919311523,
  0.7761974334716797),
 'test_time': (2.87648606300354,
  2.993947744369507,
  2.8967156410217285,
  2.9332337379455566,
  2.0765280723571777)}

In [9]:
# print out the average RMSE score for the test set
np.mean(cv_knn_basic['test_rmse'])

0.9736184607598066

In [10]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline, data, n_jobs=-1)
cv_knn_baseline

{'test_rmse': array([0.8828375 , 0.87941396, 0.88042182, 0.87632532, 0.87433672]),
 'test_mae': array([0.67463391, 0.6702135 , 0.6724693 , 0.66952366, 0.66678101]),
 'fit_time': (1.321516513824463,
  1.393965244293213,
  1.2362799644470215,
  1.0144338607788086,
  0.8633108139038086),
 'test_time': (3.69370698928833,
  3.8479959964752197,
  3.8185555934906006,
  3.9144175052642822,
  3.3873555660247803)}

In [11]:
# print out the average score for the test set
np.mean(cv_knn_baseline['test_rmse'])

0.8786670632880785

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [12]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [13]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [14]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [15]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=2.97235234775831, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [16]:
def movie_rater(userid, num, genre=""):
    rating_list = []
    
    if num > 0:
        genre0 = str(genre)
        genre = genre.lower().strip()

        ratings = {}
        i = 0
        while i < num:
            s_genre_desc = ""

            # The first step is to randomly select a movie...
            sample_movie = None
            if len(genre) > 0:
                subset = df_movies[df_movies['genres'].str.lower().str.contains(genre)]
                if len(subset) > 0:
                    sample_movie = df_movies[df_movies['genres'].str.lower().str.contains(genre)].sample(1)
                else:
                    s_genre_desc = f" tagged with genre '{genre0}'"
            else:
                sample_movie = df_movies.sample(1)

            if sample_movie is not None:
                skip = False
                while True:
                    try:
                        print(f"MOVIE {i+1}:\n{sample_movie}\n")
                        rating = input("Please rate this movie on a scale from 1 to 5, where 1 is the worst and 5 is the best... or 'n' to skip this one: ").strip().lower()
                        skip = rating=='n'
                        if not skip:
                            rating = int(rating)
                            if rating not in range(1,6):
                                raise ValueError(f"rating value must be an integer from 1 to 5")
                            d_rating = {
                                'userId': userid,
                                'movieId': sample_movie['movieId'].values[0],
                                'rating': rating
                            }
                            rating_list.append(d_rating) 
                            print(f"Thank you! Your rating has been recorded for MOVIE {i+1}:\n\t{d_rating}\n")
                            i += 1
                        break
                    except Exception as e:
                        print(f"{e}\n")
            else:
                print(f"No movies found{s_genre_desc}!")
                break

            print()
            
    return rating_list

In [17]:
# try out the new function here!
# abitrarily set user_id to 700? (anything not currently in the list of existing user_ids)
user_id = 700
n_movies_to_rate = 5
user_scifi_ratings = movie_rater(user_id, n_movies_to_rate, "sci-fi")

MOVIE 1:
      movieId                      title                    genres
6477    52885  Paprika (Papurika) (2006)  Animation|Mystery|Sci-Fi

Please rate this movie on a scale from 1 to 5, where 1 is the worst and 5 is the best... or 'n' to skip this one: n

MOVIE 1:
      movieId                                            title  \
5317     8830  Anacondas: The Hunt for the Blood Orchid (2004)   

                                      genres  
5317  Adventure|Drama|Horror|Sci-Fi|Thriller  

Please rate this movie on a scale from 1 to 5, where 1 is the worst and 5 is the best... or 'n' to skip this one: 1
Thank you! Your rating has been recorded for MOVIE 1:
	{'userId': 700, 'movieId': 8830, 'rating': 1}


MOVIE 2:
      movieId                                              title  \
5212     8500  Godzilla vs. Mechagodzilla (Gojira tai Mekagoj...   

                    genres  
5212  Action|Horror|Sci-Fi  

Please rate this movie on a scale from 1 to 5, where 1 is the worst and 5 is t

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [18]:
user_scifi_ratings

[{'userId': 700, 'movieId': 8830, 'rating': 1},
 {'userId': 700, 'movieId': 27311, 'rating': 3},
 {'userId': 700, 'movieId': 71129, 'rating': 2},
 {'userId': 700, 'movieId': 1682, 'rating': 5},
 {'userId': 700, 'movieId': 184987, 'rating': 4}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [19]:
## add the new ratings to the original ratings DataFrame
df_new_ratings = new_df.append(user_scifi_ratings, ignore_index=True)
data_new_ratings = Dataset.load_from_df(df_new_ratings[['userId', 'movieId', 'rating']], reader)
dataset_new_ratings = data_new_ratings.build_full_trainset() # comparable to X_train in other sklearn cases

In [20]:
# train a model using the new combined DataFrame
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset_new_ratings)

In [21]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movie_preds = []
for movie_id in new_df['movieId'].unique():
    movie_pred = svd.predict(user_id, movie_id)[3] # we are interested in the "est" column (index 3)
#     print(f"SVD predicts user {user_id} would rate movie {movie_id}: {movie_pred}")
    movie_preds.append((movie_id, movie_pred))

In [22]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(movie_preds, key=lambda mp: mp[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [23]:
# return the top n recommendations using the 
def recommended_movies(user_ratings, movie_title_df, n):
    print(f"based on the {n_movies_to_rate} ratings user {user_id} previously provided, I would recommend the following movies:\n")
    for recommended_movie in user_ratings[0:n]:
        movie_details = movie_title_df.query(f"movieId=={recommended_movie[0]}")
        print(f"\t{movie_details.title.values[0]} (GENRES: {movie_details.genres.values[0]})\n\t\tYour predicted rating: {recommended_movie[1]}\n")
            
recommended_movies(ranked_movies, df_movies, 5)

based on the 5 ratings user 700 previously provided, I would recommend the following movies:

	Shawshank Redemption, The (1994) (GENRES: Crime|Drama)
		Your predicted rating: 4.173811185029902

	Lawrence of Arabia (1962) (GENRES: Adventure|Drama|War)
		Your predicted rating: 4.120472240353197

	Grave of the Fireflies (Hotaru no haka) (1988) (GENRES: Animation|Drama|War)
		Your predicted rating: 4.11464784830353

	Streetcar Named Desire, A (1951) (GENRES: Drama)
		Your predicted rating: 4.104051211815716

	Wallace & Gromit: The Best of Aardman Animation (1996) (GENRES: Adventure|Animation|Comedy)
		Your predicted rating: 4.103077134442632



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.